# 使用sklearn实现关键词提取+文档余弦相似度计算

## 测试一

In [1]:
contents = [
    '我 是 中国 人。',
    '你 是 美国 人。',
    '他 叫 什么 名字？',
    '她 是 做咩啊？'
];

from sklearn.feature_extraction.text import CountVectorizer

#Convert a collection of text documents to a matrix of token counts
countVectorizer = CountVectorizer()
#Learn the vocabulary dictionary and return  term-document  matrix.
textVector = countVectorizer.fit_transform(contents)

In [2]:
textVector 

<4x5 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [3]:
textVector.todense()#看上面的输出应该是 稀疏矩阵，这里是将其转换成了 稠密矩阵，方便人为查看
#行为 document 列为term

matrix([[1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0]], dtype=int64)

In [4]:
countVectorizer.vocabulary_#输出所有的 terms和对应的下标（应该是代码里会将词映射到数字），至于为什么是两个字母及以上 看下面

{'中国': 0, '什么': 1, '做咩啊': 2, '名字': 3, '美国': 4}

In [5]:
countVectorizer.get_feature_names()

['中国', '什么', '做咩啊', '名字', '美国']

In [6]:
countVectorizer#可以从 token_pattern='(?u)\\b\\w\\w+\\b' 看到初始设定是 提取至少两个字母的词（正则表达式）

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

## 测试二

In [7]:
countVectorizer1 = CountVectorizer(
    min_df=0, 
    token_pattern=r"\b\w+\b"#现在指定 匹配至少有一个字母的term
)
textVector1 = countVectorizer1.fit_transform(contents);

In [8]:
textVector1.todense()

matrix([[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=int64)

In [9]:
countVectorizer1.vocabulary_

{'中国': 0,
 '人': 1,
 '什么': 2,
 '他': 3,
 '你': 4,
 '做咩啊': 5,
 '叫': 6,
 '名字': 7,
 '她': 8,
 '我': 9,
 '是': 10,
 '美国': 11}

In [10]:
countVectorizer1.get_feature_names()

['中国', '人', '什么', '他', '你', '做咩啊', '叫', '名字', '她', '我', '是', '美国']

## 测试三：TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
contents = [
    '我 是 中国 人。',
    '你 是 美国 人。',
    '他 叫 什么 名字？',
    '她 是 做咩啊？'
]

countVectorizer = CountVectorizer()
textVector = countVectorizer.fit_transform(contents)

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(textVector)#输入term-document  matrix，转换为稠密矩阵即是 行为 document 列为term

tfidf.toarray()

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.        ,  0.70710678,  0.        ,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ]])

In [12]:
# help(TfidfTransformer)

In [13]:
import pandas
TFIDFDataFrame = pandas.DataFrame(tfidf.toarray())
TFIDFDataFrame.columns = countVectorizer.get_feature_names()
TFIDFDataFrame

,中国,什么,做咩啊,名字,美国
0,1.0,0.000000,0.0,0.000000,0.0
1,0.0,0.000000,0.0,0.000000,1.0
2,0.0,0.707107,0.0,0.707107,0.0
3,0.0,0.000000,1.0,0.000000,0.0


In [14]:
import numpy
#对每一行进行排序，得到排序的下标
TFIDFSorted = numpy.argsort(tfidf.toarray(), axis=1)
TFIDFSorted

array([[1, 2, 3, 4, 0],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 1, 3],
       [0, 1, 3, 4, 2]], dtype=int64)

In [15]:
TFIDFDataFrame.columns # Index

Index(['中国', '什么', '做咩啊', '名字', '美国'], dtype='object')

In [16]:
TFIDFDataFrame.columns.values # array

array(['中国', '什么', '做咩啊', '名字', '美国'], dtype=object)

In [17]:
TFIDFDataFrame.columns[TFIDFSorted]

Index([['什么', '做咩啊', '名字', '美国', '中国'], ['中国', '什么', '做咩啊', '名字', '美国'],
       ['中国', '做咩啊', '美国', '什么', '名字'], ['中国', '什么', '名字', '美国', '做咩啊']],
      dtype='object')

In [18]:
TFIDFDataFrame.columns[TFIDFSorted].values #pandas.Index.values return the underlying data as an ndarray

array([['什么', '做咩啊', '名字', '美国', '中国'],
       ['中国', '什么', '做咩啊', '名字', '美国'],
       ['中国', '做咩啊', '美国', '什么', '名字'],
       ['中国', '什么', '名字', '美国', '做咩啊']], dtype=object)

In [19]:
'''if-idf矩阵中 矩阵元素值越大 则对应元素的重要性也就更加大，因此可选取值最大的前 K 个作为对应document的 K个关键词'''
import numpy
TFIDFSorted = numpy.argsort(tfidf.toarray(), axis=1)[:, -2:]
print("\n对于每条句子，得到2个关键词（从左到右重要性递增）：")
TFIDFDataFrame.columns[TFIDFSorted].values


对于每条句子，得到2个关键词（从左到右重要性递增）：


array([['美国', '中国'],
       ['名字', '美国'],
       ['什么', '名字'],
       ['美国', '做咩啊']], dtype=object)

## 测试4：实际应用

### 4.1.得到数据

In [20]:
# -*- coding: utf-8 -*-
import os
import os.path
import codecs
import pandas
import re
import jieba
import numpy

filePaths = []
fileContents = []

'''注：SogouC.mini是搜狗提供的文本分类现成语料库（有10个领域的语料，每个领域有10个样本，共有100条数据）'''
dictPath=os.path.join(os.getcwd(),'SogouC.mini\\Sample')#语料库样本所在路径

for root, dirs, files in os.walk(dictPath):
    for name in files:
        filePath = os.path.join(root, name)
        filePaths.append(filePath)
        with open(filePath, 'r', encoding='utf-8') as f:
            fileContent = f.read()
            fileContents.append(fileContent)

corpos = pandas.DataFrame({
    'filePath': filePaths, 
    'fileContent': fileContents
})

zhPattern = re.compile(u'[\u4e00-\u9fa5]+')#匹配中文的正则表达式

segments = []
filePaths = []
for index, row in corpos.iterrows():
    segments = []
    filePath = row['filePath']
    fileContent = row['fileContent']
    segs = jieba.cut(fileContent) #使用jieba进行分词   
    for seg in segs:
        if zhPattern.search(seg):#只保留中文的分词结果
            segments.append(seg)
    filePaths.append(filePath)
    row['fileContent'] = " ".join(segments)#以空格分隔分词后的结果，以适应sklearn

corpos

Building prefix dict from the default dictionary ...
Loading model from cache D:\TEMP\jieba.cache
Loading model cost 1.060 seconds.
Prefix dict has been built succesfully.


,fileContent,filePath
0,在 政府部门 明确 表示 汽车 投资 过热 产能 过剩 的 背景 下 日前 新飞 集团 却 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
1,近日 国内 一项 调查 结果表明 的 人 喜欢 开 轿车 的 人 喜欢 开 面包车 的 人 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
2,消息 传来 强制 缴纳 元 年 的 保费 最高 却 只 赔 万 随着 月 日 大限 的 临近...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
3,编辑 观点 价格 相差 两万元 我会 选 花冠 几天 前 曾 有 机会 聆听 时尚 杂志社 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
4,全球性 汽车品牌 应 如何 在 中国 做 售后服务 中国经济时报 记者 就 这 一 话题 曾...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
5,沙特 石油大臣 称 油价 到 年 将 会 稳定 沙特阿拉伯 石油 和 矿产资源 大臣 纳伊米...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
6,月 日 开摩 的 为生 的 岁 河北 邯郸 人 袁 小宝 因 拒绝 北京市 丰台区 城管 和...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
7,韩元 升值 现代 汽车 遭遇 定价 尴尬 韩国现代 汽车 似乎 进入 了 公司 发展史 上 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
8,刚刚 过去 的 五一 长假 如果 问 谁 是 将 全部 身心 都 投入 黄金周 最 放松 的...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
9,昔日 富豪 骗贷 多万 疯狂 购买 辆 豪华轿车 短短 个 月 内 获得 多万元 贷款 疯狂...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...


### 4.2.1.使用sklearn tf-idf得到文档前五个关键词
jieba.analyse.extract_tags

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

stopwords = pandas.read_csv("StopwordsCN.txt", encoding='utf8')

countVectorizer = CountVectorizer(
    stop_words=list(stopwords['stopword'].values),
    min_df=0, token_pattern=r"\b\w+\b"
)
textVector = countVectorizer.fit_transform(corpos['fileContent'])

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(textVector)

sort = numpy.argsort(tfidf.toarray(), axis=1)[:, -5:]#提取5个关键词
names = countVectorizer.get_feature_names();

keywords = pandas.Index(names)[sort].values

tagDF = pandas.DataFrame({
    'filePath':corpos.filePath, 
    'fileContent':corpos.fileContent, 
    'tag1':keywords[:, 0], 
    'tag2':keywords[:, 1], 
    'tag3':keywords[:, 2], 
    'tag4':keywords[:, 3], 
    'tag5':keywords[:, 4]
})
tagDF

,fileContent,filePath,tag1,tag2,tag3,tag4,tag5
0,在 政府部门 明确 表示 汽车 投资 过热 产能 过剩 的 背景 下 日前 新飞 集团 却 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,集团,冷藏车,奥克斯,汽车业,新飞
1,近日 国内 一项 调查 结果表明 的 人 喜欢 开 轿车 的 人 喜欢 开 面包车 的 人 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,喜欢,轿车,开,多功能,陆风
2,消息 传来 强制 缴纳 元 年 的 保费 最高 却 只 赔 万 随着 月 日 大限 的 临近...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,三者,元,保费,强制,险
3,编辑 观点 价格 相差 两万元 我会 选 花冠 几天 前 曾 有 机会 聆听 时尚 杂志社 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,车身,动力,变速箱,发动机,花冠
4,全球性 汽车品牌 应 如何 在 中国 做 售后服务 中国经济时报 记者 就 这 一 话题 曾...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,服务,中国,保养,欧美,雷克萨斯
5,沙特 石油大臣 称 油价 到 年 将 会 稳定 沙特阿拉伯 石油 和 矿产资源 大臣 纳伊米...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,石油价格,每桶,沙特,石油,纳伊米
6,月 日 开摩 的 为生 的 岁 河北 邯郸 人 袁 小宝 因 拒绝 北京市 丰台区 城管 和...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,北京市,城管,司机,出租车,黑车
7,韩元 升值 现代 汽车 遭遇 定价 尴尬 韩国现代 汽车 似乎 进入 了 公司 发展史 上 ...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,货币,韩元,汇率,现代,升值
8,刚刚 过去 的 五一 长假 如果 问 谁 是 将 全部 身心 都 投入 黄金周 最 放松 的...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,新旗云,上市,万辆,销量,奇瑞
9,昔日 富豪 骗贷 多万 疯狂 购买 辆 豪华轿车 短短 个 月 内 获得 多万元 贷款 疯狂...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,陈璋,辆,骗取,贷款,刘文辉


### 4.2.2.使用sklearn计算文档余弦相似度

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

stopwords = pandas.read_csv("StopwordsCN.txt", encoding='utf8')

countVectorizer = CountVectorizer(
    stop_words=list(stopwords['stopword'].values),
    min_df=0, token_pattern=r"\b\w+\b"
)
textVector = countVectorizer.fit_transform(corpos['fileContent'])

from sklearn.metrics import pairwise_distances #余弦相似度计算函数

distance_matrix = pairwise_distances(
    textVector, 
    metric="cosine"
)

m = 1- pandas.DataFrame(distance_matrix)
m.columns = filePaths;
m.index = filePaths;

sort = numpy.argsort(distance_matrix, axis=1)[:, 1:6]#得到相似度最相近的5个文档

similarity5 = pandas.Index(filePaths)[sort].values

similarityDF = pandas.DataFrame({
    'filePath':corpos.filePath, 
    's1': similarity5[:, 0], 
    's2': similarity5[:, 1], 
    's3': similarity5[:, 2], 
    's4': similarity5[:, 3], 
    's5': similarity5[:, 4]
})
similarityDF

,filePath,s1,s2,s3,s4,s5
0,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
1,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
2,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
3,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
4,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
5,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
6,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
7,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
8,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
9,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...,E:\Code\_githubRepositories\How-TO-Use-XXX\How...
